In [1]:
import random

# Define the fitness function
def fitness_function(x):
    return (-x**2) / 10 + 3 * x

# Function to generate a random chromosome
def generate_chromosome():
    return [random.randint(0, 1) for _ in range(5)]

# Function to decode chromosome to x value
def decode_chromosome(chromosome):
    return int(''.join(map(str, chromosome)), 2)

# Function to perform crossover between two chromosomes
def crossover(chromosome1, chromosome2):
    crossover_point = random.randint(1, len(chromosome1) - 1)
    new_chromosome1 = chromosome1[:crossover_point] + chromosome2[crossover_point:]
    new_chromosome2 = chromosome2[:crossover_point] + chromosome1[crossover_point:]
    return new_chromosome1, new_chromosome2

# Function to perform mutation on a chromosome
def mutate(chromosome):
    mutation_point = random.randint(0, len(chromosome) - 1)
    chromosome[mutation_point] = 1 - chromosome[mutation_point] # Flip the bit
    return chromosome

# Initialize the population
population_size = 10
population = [generate_chromosome() for _ in range(population_size)]

# Main loop
generation = 1
num_generations = 1000  # Maximum number of generations

while generation <= num_generations:
    # Calculate fitness values for each chromosome
    fitness_values = [fitness_function(decode_chromosome(chromosome)) for chromosome in population]
    
    # Check termination condition
    best_fitness = max(fitness_values)
    if best_fitness >= 0.9 * max(fitness_values):
        break

    # Selection - Roulette wheel selection
    selection_probabilities = [fitness / sum(fitness_values) for fitness in fitness_values]
    selected_indices = random.choices(range(population_size), weights=selection_probabilities, k=population_size)

    # Create the new population through crossover and mutation
    new_population = []
    for i in range(0, population_size, 2):
        chromosome1 = population[selected_indices[i]]
        chromosome2 = population[selected_indices[i + 1]]
        chromosome1, chromosome2 = crossover(chromosome1, chromosome2)
        if generation % 3 == 0: # Apply mutation every 3 generations
            chromosome1 = mutate(chromosome1)
            chromosome2 = mutate(chromosome2)
        new_population.extend([chromosome1, chromosome2])

    population = new_population
    generation += 1

# Print the result
best_chromosome = population[fitness_values.index(max(fitness_values))]
best_x = decode_chromosome(best_chromosome)
best_fitness = fitness_function(best_x)
print("Best Chromosome:", best_chromosome)
print("Best x:", best_x)
print("Best Fitness:", best_fitness)


Best Chromosome: [0, 1, 1, 0, 0]
Best x: 12
Best Fitness: 21.6


In [2]:
# genetic_algorithm.py

import random

# Define the fitness function
def fitness(x):
    return -x**2 / 10 + 3 * x

# Convert binary string to integer
def binary_to_int(binary_str):
    return int(binary_str, 2)

# Convert integer to binary string of length 5
def int_to_binary(x):
    return format(x, '05b')

# Selection of a parent based on fitness proportionate selection
def select_parent(population, fitness_values):
    total_fitness = sum(fitness_values)
    selection_prob = [f / total_fitness for f in fitness_values]
    return random.choices(population, weights=selection_prob, k=1)[0]

# Crossover operation
def crossover(parent1, parent2):
    crossover_point = random.randint(1, 4)
    child1 = parent1[:crossover_point] + parent2[crossover_point:]
    child2 = parent2[:crossover_point] + parent1[crossover_point:]
    return child1, child2

# Mutation operation
def mutate(chromosome):
    mutation_point = random.randint(0, 4)
    if chromosome[mutation_point] == '0':
        chromosome = chromosome[:mutation_point] + '1' + chromosome[mutation_point+1:]
    else:
        chromosome = chromosome[:mutation_point] + '0' + chromosome[mutation_point+1:]
    return chromosome

# Initial population
population_size = 10
generations = 0
max_generations = 1000
mutation_interval = 3

# Generate initial population of 10 random chromosomes
population = [int_to_binary(random.randint(0, 31)) for _ in range(population_size)]

while generations < max_generations:
    generations += 1
    
    # Evaluate fitness of the population
    fitness_values = [fitness(binary_to_int(chromosome)) for chromosome in population]
    
    # Check for termination criteria
    if max(fitness_values) >= 90:
        break
    
    # Create a new population using crossover
    new_population = []
    for _ in range(population_size // 2):
        parent1 = select_parent(population, fitness_values)
        parent2 = select_parent(population, fitness_values)
        child1, child2 = crossover(parent1, parent2)
        new_population.extend([child1, child2])
    
    # Apply mutation after every 3 generations
    if generations % mutation_interval == 0:
        new_population = [mutate(chromosome) for chromosome in new_population]
    
    population = new_population

# Final evaluation
fitness_values = [fitness(binary_to_int(chromosome)) for chromosome in population]
best_chromosome = population[fitness_values.index(max(fitness_values))]
best_x = binary_to_int(best_chromosome)
best_fitness = max(fitness_values)

print(f"Best chromosome: {best_chromosome}")
print(f"x-value: {best_x}")
print(f"Fitness value: {best_fitness}")
print(f"Generations: {generations}")


Best chromosome: 10000
x-value: 16
Fitness value: 22.4
Generations: 1000


In [5]:
import random

# Define the fitness function
def fitness_function(board):
    non_attacking_pairs = 0
    n = len(board)
    for i in range(n):
        for j in range(i + 1, n):
            if board[i] != board[j] and abs(board[i] - board[j]) != j - i:
                non_attacking_pairs += 1
    return non_attacking_pairs

# Function to generate a random board
def generate_board(n):
    return [random.randint(0, n - 1) for _ in range(n)]

# Function to perform crossover between two boards
def crossover(parent1, parent2):
    n = len(parent1)
    crossover_point = random.randint(0, n - 1)
    child1 = parent1[:crossover_point] + parent2[crossover_point:]
    child2 = parent2[:crossover_point] + parent1[crossover_point:]
    return child1, child2

# Function to perform mutation on a board
def mutate(board):
    n = len(board)
    mutation_point = random.randint(0, n - 1)
    new_value = random.randint(0, n - 1)
    board[mutation_point] = new_value
    return board

# Selection operation (Tournament Selection)
def selection(population, fitness_values):
    tournament_size = 3
    selected_indices = random.sample(range(len(population)), tournament_size)
    selected_individuals = [population[i] for i in selected_indices]
    selected_fitness = [fitness_values[i] for i in selected_indices]
    return selected_individuals[selected_fitness.index(max(selected_fitness))]

# Genetic Algorithm
def genetic_algorithm(n, pop_size, num_generations):
    population = [generate_board(n) for _ in range(pop_size)]
    for generation in range(num_generations):
        fitness_values = [fitness_function(board) for board in population]
        if max(fitness_values) == (n * (n - 1)) // 2:
            break
        new_population = []
        for _ in range(pop_size // 2):
            parent1 = selection(population, fitness_values)
            parent2 = selection(population, fitness_values)
            child1, child2 = crossover(parent1, parent2)
            if random.random() < 0.1:  # Mutation probability
                child1 = mutate(child1)
            if random.random() < 0.1:  # Mutation probability
                child2 = mutate(child2)
            new_population.extend([child1, child2])
        population = new_population
    best_board = max(population, key=fitness_function)
    return best_board, fitness_function(best_board)

# Parameters
n = 8  # Size of the board (8-Queens problem)
pop_size = 100  # Population size
num_generations = 1000  # Number of generations

# Run the genetic algorithm
best_board, best_fitness = genetic_algorithm(n, pop_size, num_generations)

# Print the results
print("Best Board:", best_board)
print("Best Fitness (non-attacking pairs):", best_fitness)

# Display the board
def print_board(board):
    n = len(board)
    for i in range(n):
        row = ['.'] * n
        row[board[i]] = 'Q'
        print(' '.join(row))

print("\nBoard Configuration:")
print_board(best_board)


Best Board: [3, 5, 7, 2, 0, 6, 4, 1]
Best Fitness (non-attacking pairs): 28

Board Configuration:
. . . Q . . . .
. . . . . Q . .
. . . . . . . Q
. . Q . . . . .
Q . . . . . . .
. . . . . . Q .
. . . . Q . . .
. Q . . . . . .


In [4]:
import random
import numpy as np

# Define the locations
locations = ['MM Alam Road', 'Johar Town', 'Shahdara', 'DHA Phase 6', 'Wapda Town', 'Askari 10', 'Allama Iqbal Town', 'Mall Road']

# Generate a random distance matrix (symmetric and diagonal is zero)
def generate_distance_matrix(size):
    matrix = np.random.randint(0, 50, size=(size, size))
    matrix = (matrix + matrix.T) / 2
    np.fill_diagonal(matrix, 0)
    return matrix

distance_matrix = generate_distance_matrix(len(locations))

# Define the fitness function (total distance of the route)
def fitness_function(route):
    distance = 0
    for i in range(len(route)):
        distance += distance_matrix[route[i], route[(i + 1) % len(route)]]
    return -distance  # Use negative distance because we want to minimize it

# Generate an initial population
def generate_population(pop_size, num_locations):
    population = []
    for _ in range(pop_size):
        individual = list(range(num_locations))
        random.shuffle(individual)
        population.append(individual)
    return population

# Crossover operation
def crossover(parent1, parent2):
    size = len(parent1)
    start, end = sorted(random.sample(range(size), 2))
    child1 = [None] * size
    child1[start:end] = parent1[start:end]
    child2 = [item for item in parent2 if item not in child1[start:end]]
    child1 = [item if item is not None else child2.pop(0) for item in child1]
    return child1

# Mutation operation
def mutate(individual):
    i, j = random.sample(range(len(individual)), 2)
    individual[i], individual[j] = individual[j], individual[i]
    return individual

# Selection operation (Roulette Wheel Selection)
def selection(population, fitness_values):
    total_fitness = sum(fitness_values)
    selection_prob = [f / total_fitness for f in fitness_values]
    selected_indices = random.choices(range(len(population)), weights=selection_prob, k=2)
    return population[selected_indices[0]], population[selected_indices[1]]

# Genetic Algorithm
def genetic_algorithm(pop_size, num_locations, num_generations):
    population = generate_population(pop_size, num_locations)
    for generation in range(num_generations):
        fitness_values = [fitness_function(individual) for individual in population]
        new_population = []
        for _ in range(pop_size):
            parent1, parent2 = selection(population, fitness_values)
            child = crossover(parent1, parent2)
            if random.random() < 0.1:  # Mutation probability
                child = mutate(child)
            new_population.append(child)
        population = new_population
    best_individual = max(population, key=fitness_function)
    return best_individual, -fitness_function(best_individual)

# Parameters
pop_size = 100
num_generations = 500
num_locations = len(locations)

# Run the genetic algorithm
best_route, best_distance = genetic_algorithm(pop_size, num_locations, num_generations)

# Print the results
best_route_locations = [locations[i] for i in best_route]
print("Best Route:", best_route_locations)
print("Best Distance:", best_distance)

# Display the distance matrix
print("\nDistance Matrix:")
print(distance_matrix)


Best Route: ['Wapda Town', 'MM Alam Road', 'Johar Town', 'DHA Phase 6', 'Allama Iqbal Town', 'Mall Road', 'Shahdara', 'Askari 10']
Best Distance: 119.5

Distance Matrix:
[[ 0.   9.  37.5 16.  22.5 16.  17.   3. ]
 [ 9.   0.  19.  25.  21.5 33.  35.  12.5]
 [37.5 19.   0.  47.5 33.  17.  15.5 16.5]
 [16.  25.  47.5  0.  31.  24.5 15.5 17.5]
 [22.5 21.5 33.  31.   0.   8.  24.  30. ]
 [16.  33.  17.  24.5  8.   0.  17.  43.5]
 [17.  35.  15.5 15.5 24.  17.   0.   6. ]
 [ 3.  12.5 16.5 17.5 30.  43.5  6.   0. ]]
